In [1]:
import os
import h5py
import json
import glob
import shutil

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
import pandas as pd

import gizmo_analysis as ga
import utilities as ut

%matplotlib inline

mpl.rc('font', size=20)
mpl.rc('figure', figsize=(8, 5), facecolor='w')
mpl.rcParams['font.family'] = 'STIXGeneral'
mpl.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'

In [2]:
table_fn = 'accretion_history/stars_accretion_history_m12f_v1.csv'
table = pd.read_csv(table_fn)
indices = table['id_stars'].to_numpy()

In [3]:
# Picking a sample simulation and a snapshot. 
sim_dir = f'/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12f_res7100'
nsnap = 600

#Read using gizmo
part = ga.io.Read.read_snapshots(
    species=['star'], snapshot_values=nsnap, simulation_directory=sim_dir,
    assign_hosts=True, assign_hosts_rotation=True)


# in utilities.simulation.Snapshot():
* reading:  scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12f_res7100/snapshot_times.txt

  using snapshot index = 600, redshift = 0.000


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12f_res7100/output/snapdir_600/snapshot_600.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 96239056 particles
    dark2     (id = 2): 7591203 particles
    gas       (id = 0): 80307825 particles
    star      (id = 4): 16066489 particles
    blackhole (id = 5): 0 particles

* reading species: ['star']
* reading particles from:
    snapshot_600.0.hdf5
    snapshot_600.1.hdf5
    snapshot_600.2.hdf5
    snapshot_600.3.hdf5

* reading cosmological parameters from:  scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12f_res7100/initial_condition/ic_agora_m12f.conf

* checking sanity of particle properties
! warning: star id.child [min, max] = [0.000,

In [4]:
# Get desired attributes to add 
form_scalefactor = part['star']['form.scalefactor'][indices]
form_time = part['star'].prop('form.time')[indices]
cyl_velocity = part['star'].prop('host.velocity.principle.cylindrical')[indices]
cyl_distance = part['star'].prop('host.distance.principle.cylindrical')[indices]
velocity = part['star'].prop('host.velocity.principle')[indices]
distance = part['star'].prop('host.distance.principle')[indices]

In [5]:
table['form_scalefactor'] = form_scalefactor
table['form_time'] = form_time
table['vr_cyl'] = cyl_velocity[:, 0]
table['vphi_cyl'] = cyl_velocity[:, 1]
table['vz_cyl'] = cyl_velocity[:, 2]
table['r_cyl'] = cyl_distance[:, 0]
table['phi_cyl'] = cyl_distance[:, 1]
table['z_cyl'] = cyl_distance[:, 2]
table['vx_cart'] = velocity[:, 0]
table['vy_cart'] = velocity[:, 1]
table['vz_cart'] = velocity[:, 2]
table['x_cart'] = distance[:, 0]
table['y_cart'] = distance[:, 1]
table['z_cart'] = distance[:, 2]

In [6]:
table.to_csv('accretion_history/stars_accretion_history_m12f_v2.csv')